# Install protinfer 

Note protinfer was not re-trained so may be slightly biased.

Protinfer needs to be installed as per the guidelines and then the following command (then CARE will take care of the rest!)

```
git clone https://github.com/google-research/proteinfer
cd ~/proteinfer
pip3 install -r requirements.txt
python3 install_models.py
```

Example to run (you don't need this for our benchmarking pipeline but it may be useful to know!)

```
python3 proteinfer.py -i price.fasta -o price.tsv
```


## Run CARE

In [1]:
import sys
sys.path.append('/disk1/ariane/vscode/CARE/')
sys.path.append('/disk1/ariane/vscode/CARE/CARE/')

from CARE.processing import *
from CARE.task1 import *

data_dir = '/disk1/ariane/vscode/CARE/data/task1/'
output_dir = '/disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/'

In [2]:
# Save in the required format
# Make a task1
tasker = Task1(data_folder=data_dir, output_folder=output_dir)
for split in ['30', '30-50', 'price', 'promiscuous']:
    # For proteInfer you need to point where it was saved.
    tasker.get_proteinfer(split, proteinfer_dir='/disk1/ariane/pycharm/CARE/proteinfer/', save=True)

--------------------------------------------------------------------------------
        Warning! You must already have the proteInfer environment made	         
--------------------------------------------------------------------------------
/disk1/ariane/vscode/CARE/data/task1/30_protein_test.fasta


## Train the model
Future work may include training the model:
```
python train.py --data_base_path=./testdata/ \
--label_vocab_path=./data/vocabs/EC.tsv \
--hparams_set=small_test_model \
--output_dir=trained_baseline1
```